In [ ]:
!pip install tiktoken chromadb

In [1]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

raw_docs = [WebBaseLoader(url).load() for url in urls]
docs = [doc for raw_doc in raw_docs for doc in raw_doc]

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap=0)
doc_chunks = splitter.split_documents(docs)
len(doc_chunks)

239

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

vec_db = Chroma.from_documents(
    documents=doc_chunks,
    collection_name="rag",
    embedding=OllamaEmbeddings(model="qwen2.5-coder:14b")
)
retriever = vec_db.as_retriever()

In [4]:
question = "What security issue could we face while using the LLM ?"

In [6]:
# TEST
relevant_docs = retriever.invoke(question)
for relevant_doc in relevant_docs:
    print("==> ", relevant_doc.metadata['source'], relevant_doc.page_content[:80])

==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ One simple and intuitive way to defend the model against adversarial attacks is 
==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ Fig. 13. UI for humans to do tool-assisted adversarial attack on a classifier. H
==>  https://lilianweng.github.io/posts/2023-06-23-agent/ }
]
Challenges#
After going through key ideas and demos of building LLM-centered
==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ In the white-box setting, we have full access to the model parameters and archit


In [9]:
from pydantic import BaseModel, Field

class CheckDocs(BaseModel):
    is_relevant: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

In [10]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen2.5-coder:14b", temperature=0)
llm_structured = llm.with_structured_output(CheckDocs)

In [12]:
from langchain_core.prompts import ChatPromptTemplate

system_instruction = """Return only a 'yes' or 'no' depends on whether the document is relevant to the question"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_instruction),
        ("human", "Document: \n {doc} \n\nQuestion: {question}"),
    ]
)
retriv_eval = prompt | llm_structured

In [14]:
# TEST
for relevant_doc in relevant_docs:
    content = relevant_doc.page_content
    print("is_relevant: ", retriv_eval.invoke({"doc": content, "question": question}), "==> ", relevant_doc.metadata['source'], relevant_doc.page_content[:80])

is_relevant:  is_relevant='yes' ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ One simple and intuitive way to defend the model against adversarial attacks is 
is_relevant:  is_relevant='yes' ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ Fig. 13. UI for humans to do tool-assisted adversarial attack on a classifier. H
is_relevant:  is_relevant='no' ==>  https://lilianweng.github.io/posts/2023-06-23-agent/ }
]
Challenges#
After going through key ideas and demos of building LLM-centered
is_relevant:  is_relevant='yes' ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ In the white-box setting, we have full access to the model parameters and archit


In [20]:
def concat_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [23]:
from langchain_core.output_parsers import StrOutputParser

gen_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:"
),
    ]
)
gen_resp = gen_prompt | llm | StrOutputParser()

In [26]:
# TEST
resp = gen_resp.invoke({"context": concat_docs(relevant_docs), "question": question})
print(resp)

While using LLMs, one significant security issue is the risk of adversarial attacks, where malicious inputs are crafted to manipulate model outputs. Defending against these attacks by instructing models to avoid harmful content can reduce their success rate but may also lead to decreased general quality and potential misinterpretation of instructions in certain scenarios.
